In [ ]:
# Package to have access to the Zindi Platform features
!pip -q install git+https://github.com/eaedk/testing-zindi-package.git

In [ ]:
from zindi.user import Zindian
user = Zindian(username="CalebEmelike")

Your password
>> ··········

[ 🟢 ] 👋🏾👋🏾 Welcome CalebEmelike 👋🏾👋🏾



In [ ]:
user.select_a_challenge(reward='all', kind='hackathon', active='true')

__________________________________________________________________________________________________________________________________
|     |              |                  |                    |          
|index|  challenge   |     problem      |       reward       |    id    
|     |              |                  |                    |          
----------------------------------------------------------------------------------------------------------------------------------
|  0  | Public Hack  |    Prediction    |      $300 USD      | zindi-weekendz-laduma-analytics-football-league-wi...
----------------------------------------------------------------------------------------------------------------------------------
|  1  | Public Hack  |  Classification  |      $300 USD      | kenyan-sign-language-classification-hackathon...
----------------------------------------------------------------------------------------------------------------------------------
|  2  | Public Hack  |      

In [ ]:
user.download_dataset(destination="football")

football/VariableDefinitions.csv: 100%|██████████| 1.80k/1.80k [00:00<00:00, 844ko/s]
football/Train.csv: 100%|██████████| 23.7k/23.7k [00:00<00:00, 3.17Mo/s]
football/Test.csv: 100%|██████████| 11.4k/11.4k [00:00<00:00, 4.78Mo/s]
football/SampleSubmission.csv: 100%|██████████| 3.90k/3.90k [00:00<00:00, 1.73Mo/s]
football/Starter_Notebook.ipynb: 100%|██████████| 166k/166k [00:00<00:00, 5.34Mo/s]
football/game_statistics.csv: 100%|██████████| 328M/328M [00:15<00:00, 21.5Mo/s]


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.3 MB/s 


In [ ]:
!pip install -U xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 192.9 MB 56 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
!pip install -U lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 12.6 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [ ]:
import numpy as np
import pandas as pd
import ast

from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import Ridge ,LinearRegression, LogisticRegression

from sklearn.model_selection import KFold,StratifiedKFold ,GroupKFold, cross_val_score, train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

from sklearn.cluster import KMeans

pd.set_option('max_column', 10000)
pd.set_option('max_row', 10000)

# Plot Packages
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('/content/football/Train.csv')
test = pd.read_csv('/content/football/Test.csv')
ss = pd.read_csv('/content/football/SampleSubmission.csv')
variable = pd.read_csv('/content/football/VariableDefinitions.csv')
game = pd.read_csv('/content/football/game_statistics.csv')

In [ ]:
game['possession_diff'] = game['End_minutes']-game['Start_minutes']

In [ ]:
per_game = game.groupby('Game_ID').agg(count_games=('Game_ID', 'count'), n_player_game=('Player_ID', 'nunique'),                                       
             game_shoot_mean = ('Shots', 'mean'),
             game_shoot_sum = ('Shots', 'sum'),
             game_sot_mean = ('SoT', 'mean'),
             game_sot_sum = ('SoT', 'sum'),
             game_acc_pass_mean = ('Accurate passes', 'mean'),
             game_acc_pass_sum = ('Accurate passes', 'sum'),
             game_pass_mean = ('Passes', 'mean'),
             game_pass_sum = ('Passes', 'sum'),  
             game_X_mean = ('X', 'mean'),
             game_X_min = ('X', 'min'),
             game_X_max = ('X', 'max'),  
             game_Y_mean = ('Y', 'mean'),
             game_Y_min = ('Y', 'min'),
             game_Y_max = ('Y', 'max'),                                     
             ).reset_index()

In [ ]:
per_team = game.groupby(['Game_ID','Team']).agg(count_team=('Team', 'count'), n_player_team=('Player_ID', 'nunique'), n_manager_team=('Manager', 'nunique'),                                   
             team_shoot_mean = ('Shots', 'mean'),
             team_shoot_sum = ('Shots', 'sum'),
             team_shoot_min = ('Shots', 'min'),
             team_shoot_max = ('Shots', 'max'),
             team_sot_mean = ('SoT', 'mean'),
             team_sot_sum = ('SoT', 'sum'),
             team_sot_min = ('SoT', 'min'),
             team_sot_max = ('SoT', 'max'),
             team_acc_pass_mean = ('Accurate passes', 'mean'),
             team_acc_pass_sum = ('Accurate passes', 'sum'),
             team_pass_mean = ('Passes', 'mean'),
             team_pass_sum = ('Passes', 'sum'),
             team_X_mean = ('X', 'mean'),
             team_X_min = ('X', 'min'),
             team_X_max = ('X', 'max'),  
             team_Y_mean = ('Y', 'mean'),
             team_Y_min = ('Y', 'min'),
             team_Y_max = ('Y', 'max'),  
             team_possession_sum = ('possession_diff', 'sum'),
             team_possession_mean = ('possession_diff', 'mean'), 
             team_possession_min = ('possession_diff', 'min'),
             team_possession_max = ('possession_diff', 'max'),                                      
             ).reset_index()

In [ ]:
per_opp1 = game.groupby(['Opposition_Team']).agg(                                   
             opp1_shoot_mean = ('Shots', 'mean'),
             opp1_shoot_sum = ('Shots', 'sum'),
             opp1_sot_mean = ('SoT', 'mean'),
             opp1_sot_sum = ('SoT', 'sum'),
             opp1_acc_pass_mean = ('Accurate passes', 'mean'),
             opp1_acc_pass_sum = ('Accurate passes', 'sum'),
             opp1_pass_mean = ('Passes', 'mean'),
             opp1_pass_sum = ('Passes', 'sum'),
             opp1_X_mean = ('X', 'mean'),
             opp1_X_min = ('X', 'min'),
             opp1_X_max = ('X', 'max'),  
             opp1_Y_mean = ('Y', 'mean'),
             opp1_Y_min = ('Y', 'min'),
             opp1_Y_max = ('Y', 'max'),                                    
             ).reset_index()

In [ ]:
per_team1 = game.groupby(['Team']).agg(                                   
             team1_shoot_mean = ('Shots', 'mean'),
             team1_shoot_sum = ('Shots', 'sum'),
             team1_sot_mean = ('SoT', 'mean'),
             team1_sot_sum = ('SoT', 'sum'),
             team1_acc_pass_mean = ('Accurate passes', 'mean'),
             team1_acc_pass_sum = ('Accurate passes', 'sum'),
             team1_pass_mean = ('Passes', 'mean'),
             team1_pass_sum = ('Passes', 'sum'),
             team1_X_mean = ('X', 'mean'),
             team1_X_min = ('X', 'min'),
             team1_X_max = ('X', 'max'),  
             team1_Y_mean = ('Y', 'mean'),
             team1_Y_min = ('Y', 'min'),
             team1_Y_max = ('Y', 'max'),                                    
             ).reset_index()

In [ ]:
len_train = len(train)
df = pd.concat([train, test]).reset_index(drop=True)

In [ ]:
df.head()

,Date,Season,Match_ID,Game_ID,Home Team,Away Team,Score
0,2017-02-15,1,1.0,ID_KAG4KAE9,Antennae,Andromeda,Away win
1,2016-09-14,1,12.0,ID_T7H1XN5Z,Andromeda,Antennae,Draw
2,2016-12-12,1,13.0,ID_O1P9FGN4,Andromeda,Butterfly,Away win
3,2017-01-25,1,14.0,ID_TR5M4KQV,Andromeda,Cigar,Away win
4,2016-08-24,1,15.0,ID_EOIECRE2,Andromeda,Circinus,Home Win


In [ ]:
per_team.rename(columns={'Team':'Home Team'}, inplace=True)

per_team1.rename(columns={'Team':'Home Team'}, inplace=True)
per_opp1.rename(columns={'Opposition_Team':'Away Team'}, inplace=True)

In [ ]:
df = df.merge(per_game, on='Game_ID', how='left')

In [ ]:
df = df.merge(per_team, on=['Game_ID','Home Team'], how='left')

In [ ]:
df = df.merge(per_opp1, on=['Away Team'], how='left')
df = df.merge(per_team1, on=['Home Team'], how='left')

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year
df['year'] = df['year'].astype(float)
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day

In [ ]:
cat_col = ['Home Team','Away Team']

for i in cat_col:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i])

In [ ]:
train_df = df[:len_train]
test_df = df[len_train:]

In [ ]:
train_df['Score'] = train_df['Score'].replace({'Home Win':0, 'Draw':1, 'Away win':2})

In [ ]:
X = train_df.drop(columns=['Game_ID','Date','Score'])
y = train_df['Score']
tes = test_df.drop(columns=['Game_ID','Date','Score'])

In [ ]:
%%time
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=2022)

oofs = np.zeros((len(train_df)))
preds = np.zeros((len(tes)))
predic = []

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
    
    X_trn, y_trn = X.iloc[trn_idx], y.iloc[trn_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    
    
    clf = XGBClassifier(learning_rate=0.05,   max_depth=6,  n_estimators=1000)
    clf.fit(X_trn, y_trn, eval_set = [(X_val, y_val)],verbose =200, early_stopping_rounds=200)
    
    vp = clf.predict(X_val)
    val_score = accuracy_score((y_val), (vp))
    
    print(f'\nFold {fold_+1} Val score: {val_score}\n')
    
    tp = clf.predict(tes)
    
    predic.append(val_score)
    
    
    oofs[val_idx] = vp
    preds += tp/10

oof_score = accuracy_score((y), (oofs))
print(f'\nOOF ACC score is : {oof_score}')

[0]	validation_0-mlogloss:1.08021
[200]	validation_0-mlogloss:0.99194
[261]	validation_0-mlogloss:1.03557

Fold 1 Val score: 0.5952380952380952

[0]	validation_0-mlogloss:1.08699
[200]	validation_0-mlogloss:1.03398
[279]	validation_0-mlogloss:1.07272

Fold 2 Val score: 0.5476190476190477

[0]	validation_0-mlogloss:1.08509
[200]	validation_0-mlogloss:1.06949
[261]	validation_0-mlogloss:1.11795

Fold 3 Val score: 0.5853658536585366

[0]	validation_0-mlogloss:1.08638
[200]	validation_0-mlogloss:1.23795
[228]	validation_0-mlogloss:1.25646

Fold 4 Val score: 0.5121951219512195

[0]	validation_0-mlogloss:1.09006
[200]	validation_0-mlogloss:0.94283
[278]	validation_0-mlogloss:0.99716

Fold 5 Val score: 0.7073170731707317

[0]	validation_0-mlogloss:1.08391
[200]	validation_0-mlogloss:0.79806
[305]	validation_0-mlogloss:0.87832

Fold 6 Val score: 0.6097560975609756

[0]	validation_0-mlogloss:1.08082
[200]	validation_0-mlogloss:0.80341
[277]	validation_0-mlogloss:0.82605

Fold 7 Val score: 0.634

In [ ]:
%%time
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=2022)

oofs_1 = np.zeros((len(train_df)))
preds_1 = np.zeros((len(tes)))
predic = []

# grp = train['UserID']

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
    
    X_trn, y_trn = X.iloc[trn_idx], y.iloc[trn_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    
    
    clf = CatBoostClassifier(learning_rate=0.05,  depth=7,   n_estimators=200)
    
    clf.fit(X_trn, y_trn, eval_set = [(X_val, y_val)],verbose =200, early_stopping_rounds=100)
    
    vp = clf.predict(X_val)[:,0]
    val_score = accuracy_score((y_val), (vp))
    
    print(f'\nFold {fold_+1} Val score: {val_score}\n')
    
    tp = clf.predict(tes)[:,0]
    
    predic.append(val_score)
    
    
    oofs_1[val_idx] = vp
    preds_1 += tp/10

oof_score = accuracy_score((y), (oofs_1))
print(f'\nOOF ACC score is : {oof_score}')

0:	learn: 1.0878772	test: 1.0937768	best: 1.0937768 (0)	total: 109ms	remaining: 21.7s
199:	learn: 0.3317237	test: 0.9042783	best: 0.9042055 (187)	total: 6.3s	remaining: 0us

bestTest = 0.9042055105
bestIteration = 187

Shrink model to first 188 iterations.

Fold 1 Val score: 0.6190476190476191

0:	learn: 1.0871884	test: 1.0852910	best: 1.0852910 (0)	total: 40.9ms	remaining: 8.14s
199:	learn: 0.3413447	test: 0.8969733	best: 0.8923567 (173)	total: 6.05s	remaining: 0us

bestTest = 0.8923567205
bestIteration = 173

Shrink model to first 174 iterations.

Fold 2 Val score: 0.6190476190476191

0:	learn: 1.0872811	test: 1.0952158	best: 1.0952158 (0)	total: 40.7ms	remaining: 8.1s
199:	learn: 0.3267365	test: 1.0070924	best: 0.9979720 (113)	total: 6.02s	remaining: 0us

bestTest = 0.9979720343
bestIteration = 113

Shrink model to first 114 iterations.

Fold 3 Val score: 0.4878048780487805

0:	learn: 1.0868131	test: 1.0948828	best: 1.0948828 (0)	total: 40.4ms	remaining: 8.05s
199:	learn: 0.3319547	

In [ ]:
xgb_oof = pd.DataFrame(oofs, columns=['xgb'])
xgb_pred = pd.DataFrame(preds, columns=['xgb'])

lgb_oof = pd.DataFrame(oofs_1, columns=['cat'])
lgb_pred = pd.DataFrame(preds_1, columns=['cat'])

In [ ]:
a = X[['team_possession_sum','team_possession_mean','team_possession_min','team_possession_max']]
b = tes[['team_possession_sum','team_possession_mean','team_possession_min','team_possession_max']]

In [ ]:
b.reset_index(drop=True, inplace=True)

In [ ]:
new_X = pd.concat([xgb_oof, lgb_oof, a], 1)
new_tes = pd.concat([xgb_pred, lgb_pred, b], 1)

In [ ]:
new_tes.fillna(-999, inplace=True)

In [ ]:
%%time
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=2022)

oofs_2 = np.zeros((len(train_df)))
preds_2 = np.zeros((len(tes)))
predic = []

for fold_, (trn_idx, val_idx) in enumerate(folds.split(new_X, y)):
    
    X_trn, y_trn = new_X.iloc[trn_idx], y.iloc[trn_idx]
    X_val, y_val = new_X.iloc[val_idx], y.iloc[val_idx]
    
    
    clf = LogisticRegression()
    
    clf.fit(new_X, y)
    
    vp = clf.predict(X_val)
    val_score = accuracy_score((y_val), (vp))
    
    print(f'\nFold {fold_+1} Val score: {val_score}\n')
    
    tp = clf.predict(new_tes)
    
    predic.append(val_score)
    
    
    oofs_2[val_idx] = vp
    preds_2 += tp/10

oof_score = accuracy_score((y), (oofs_2))
print(f'\nOOF ACC score is : {oof_score}')


Fold 1 Val score: 0.5476190476190477


Fold 2 Val score: 0.5952380952380952


Fold 3 Val score: 0.5609756097560976


Fold 4 Val score: 0.5365853658536586


Fold 5 Val score: 0.6341463414634146


Fold 6 Val score: 0.6341463414634146


Fold 7 Val score: 0.6097560975609756


Fold 8 Val score: 0.7073170731707317


Fold 9 Val score: 0.6585365853658537


Fold 10 Val score: 0.5853658536585366


OOF ACC score is : 0.6067961165048543
CPU times: user 483 ms, sys: 8.17 ms, total: 492 ms
Wall time: 493 ms


In [ ]:
np.mean(predic)

0.6191637630662019

In [ ]:
d = {'Game_ID':test['Game_ID'], 'Score':preds_2}
submit = pd.DataFrame(d)

In [ ]:
submit.head()

,Game_ID,Score
0,ID_8518U587,2.0
1,ID_H49BIKG7,0.0
2,ID_PO6SP4VA,0.0
3,ID_MZRCNBAQ,2.0
4,ID_CV9VOLIU,0.0


In [ ]:
submit['Score'] = submit['Score'].round()

In [ ]:
submit['Score'] = submit['Score'].replace({0:'Home Win', 1:'Draw', 2:'Away win'})

In [ ]:
submit['Score'].value_counts()

Home Win    136
Away win     49
Draw         49
Name: Score, dtype: int64

In [ ]:
submit.to_csv('Final_football.csv', index=False)